In [2]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/Colab_Models/phi3p5_lora

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -q torch transformers datasets peft accelerate bitsandbytes py7zr

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 12.7 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# ...
print(torch.cuda.get_device_name(0))

Tesla T4


In [5]:
#Load base model
model_id = "microsoft/phi-3.5-mini-instruct"  # (example)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Adjust target_modules to match phi-3.5 layering
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[ "q_proj",
            "k_proj",
            "v_proj",
            "o_proj"],  # confirm correct layer names
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.use_cache = False


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
# If you have a direct link or a file in your GDrive:
# !wget -O airbnb_data.csv "https://drive.google.com/uc?export=download&id=<YOUR_FILE_ID>"

# Or if you have it in your local Colab filesystem already, just reference its path:
file_path = "https://drive.google.com/uc?export=download&id=17uGXWjrjn_o68K5xuEd0vxCVMT4lYWV0"
#!wget -O airbnb_tabular.csv
from datasets import load_dataset

raw_data = load_dataset("csv", data_files=file_path)
dataset = raw_data["train"]   # Single split named "train" by default

print("Number of samples in dataset:", len(dataset))
print("Columns:", dataset.column_names)
print("Sample row:", dataset[0])



Generating train split: 0 examples [00:00, ? examples/s]

Number of samples in dataset: 22570
Columns: ['Unnamed: 0', 'id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30',

In [7]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Choose your model ID (example):
model_id = "microsoft/phi-3.5-mini-instruct"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print("Loading base model in 4-bit precision...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,  # Some Phi models require this
    device_map="auto"        # Automatic GPU mapping
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # ensure there's a pad token

# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    # You must confirm the correct layer names for phi-3.5:
    # This is just an example; "query_key_value" might be correct,
    # but "Wqkv", "fc1", "fc2" might be for phi-2
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Prepare model for 4-bit + LoRA
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# By default, we disable caching to avoid certain issues with 4-bit + LoRA
model.config.use_cache = False

print("Model and LoRA adapters are ready.")


Loading base model in 4-bit precision...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and LoRA adapters are ready.


4) Prompt Engineering

In [8]:
def create_prompt_style_a(description, original_title):
    """
    Style A:
    More direct, instructive prompt.
    """
    prompt = f"""
    Generate an appealing Airbnb listing title based on the description below.

    Description:
    {description}

    Title: {original_title}
    """
    return prompt.strip()

def create_prompt_style_b(description, original_title):
    """
    Style B:
    A 'creative copywriter' style prompt with a different phrasing.
    """
    prompt = f"""
    You are a creative copywriter.
    Write a short, catchy Airbnb listing title that captures the essence of:

    {description}

    Answer:
    {original_title}
    """
    return prompt.strip()


5) Tokenize the Dataset (Style A)

In [9]:
def tokenize_style_a(examples):
    text = create_prompt_style_a(
        examples["description"],
        examples["name"]
    )
    # You can adjust max_length as needed
    return tokenizer(text, truncation=True, max_length=512)

# Apply to entire dataset
dataset_style_a = dataset.map(
    tokenize_style_a,
    remove_columns=dataset.column_names,
    batched=False
)

print(dataset_style_a[0])

Map:   0%|          | 0/22570 [00:00<?, ? examples/s]

{'input_ids': [3251, 403, 385, 5929, 12818, 5593, 29890, 9877, 18028, 3611, 2729, 373, 278, 6139, 2400, 29889, 13, 13, 1678, 12953, 29901, 13, 1678, 1619, 11785, 3765, 6592, 8345, 411, 263, 2919, 3474, 756, 263, 26681, 287, 11223, 29991, 739, 13016, 2197, 23994, 697, 470, 1023, 322, 338, 1644, 29878, 635, 5982, 925, 1023, 10930, 515, 383, 1144, 11059, 4815, 29889, 1174, 2212, 29891, 2107, 12374, 1934, 297, 278, 4038, 322, 4780, 2130, 304, 4780, 8608, 23131, 267, 29892, 22983, 322, 3593, 267, 29889, 14525, 583, 322, 4344, 310, 599, 24646, 526, 12853, 29889, 450, 2913, 15043, 7569, 650, 29892, 306, 29915, 29885, 27032, 590, 12355, 873, 3765, 6592, 8345, 297, 383, 1144, 11059, 4815, 4038, 313, 8028, 29871, 29906, 29897, 363, 1235, 297, 263, 7258, 263, 8076, 29889, 887, 674, 6232, 278, 263, 8076, 411, 592, 322, 372, 338, 8072, 15252, 3276, 411, 263, 1583, 274, 1008, 292, 29181, 29889, 7803, 2305, 508, 5948, 8709, 1532, 408, 278, 5716, 756, 263, 26624, 2159, 6592, 29889, 306, 884, 505, 263,

6) Fine Tuning for Style A

In [10]:
output_dir_a = "phi3p5_lora_styleA"

training_args = TrainingArguments(
    output_dir=output_dir_a,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_steps=400,           # or use num_train_epochs if you prefer
    save_steps=100,
    logging_steps=10,
    fp16=True,
    # 'paged_adamw_8bit' is used in certain PEFT examples,
    # but you could also just do "adamw_hf"
    optim="paged_adamw_8bit",
    # If you want to push to HF Hub, you can set push_to_hub=True, etc.
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_style_a,
    data_collator=data_collator,
)

trainer.train()

# Optional: Save final adapter
trainer.save_model(output_dir_a)
tokenizer.save_pretrained(output_dir_a)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,10.357600
20,9.442500
30,8.796400
40,8.499300
50,8.339900
60,8.404000
70,8.192600
80,8.244200
90,8.286500
100,8.282400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

('phi3p5_lora_styleA/tokenizer_config.json',
 'phi3p5_lora_styleA/special_tokens_map.json',
 'phi3p5_lora_styleA/tokenizer.model',
 'phi3p5_lora_styleA/added_tokens.json',
 'phi3p5_lora_styleA/tokenizer.json')

7) Tokenize the Dataset (Style B)

In [11]:
def tokenize_style_b(examples):
    text = create_prompt_style_b(
        examples["description"],
        examples["name"]
    )
    # You can adjust max_length as needed
    return tokenizer(text, truncation=True, max_length=512)

# Apply to entire dataset
dataset_style_b = dataset.map(
    tokenize_style_b,
    remove_columns=dataset.column_names,
    batched=False
)

print(dataset_style_b[0])

Map:   0%|          | 0/22570 [00:00<?, ? examples/s]

{'input_ids': [887, 526, 263, 907, 1230, 3509, 13236, 29889, 13, 1678, 14350, 263, 3273, 29892, 4380, 29891, 5593, 29890, 9877, 18028, 3611, 393, 4332, 1973, 278, 3686, 663, 310, 29901, 13, 13, 1678, 1619, 11785, 3765, 6592, 8345, 411, 263, 2919, 3474, 756, 263, 26681, 287, 11223, 29991, 739, 13016, 2197, 23994, 697, 470, 1023, 322, 338, 1644, 29878, 635, 5982, 925, 1023, 10930, 515, 383, 1144, 11059, 4815, 29889, 1174, 2212, 29891, 2107, 12374, 1934, 297, 278, 4038, 322, 4780, 2130, 304, 4780, 8608, 23131, 267, 29892, 22983, 322, 3593, 267, 29889, 14525, 583, 322, 4344, 310, 599, 24646, 526, 12853, 29889, 450, 2913, 15043, 7569, 650, 29892, 306, 29915, 29885, 27032, 590, 12355, 873, 3765, 6592, 8345, 297, 383, 1144, 11059, 4815, 4038, 313, 8028, 29871, 29906, 29897, 363, 1235, 297, 263, 7258, 263, 8076, 29889, 887, 674, 6232, 278, 263, 8076, 411, 592, 322, 372, 338, 8072, 15252, 3276, 411, 263, 1583, 274, 1008, 292, 29181, 29889, 7803, 2305, 508, 5948, 8709, 1532, 408, 278, 5716, 756,

In [12]:
output_dir_b = "phi3p5_lora_styleB"

training_args = TrainingArguments(
    output_dir=output_dir_b,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_steps=400,           # or use num_train_epochs if you prefer
    save_steps=100,
    logging_steps=10,
    fp16=True,
    # 'paged_adamw_8bit' is used in certain PEFT examples,
    # but you could also just do "adamw_hf"
    optim="paged_adamw_8bit",
    # If you want to push to HF Hub, you can set push_to_hub=True, etc.
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_style_b,
    data_collator=data_collator,
)

trainer.train()

# Optional: Save final adapter
trainer.save_model(output_dir_b)
tokenizer.save_pretrained(output_dir_b)

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,8.522600
20,7.856500
30,7.759600
40,7.604600
50,7.586500
60,7.655000
70,7.452800
80,7.572500
90,7.626700
100,7.637700


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

('phi3p5_lora_styleB/tokenizer_config.json',
 'phi3p5_lora_styleB/special_tokens_map.json',
 'phi3p5_lora_styleB/tokenizer.model',
 'phi3p5_lora_styleB/added_tokens.json',
 'phi3p5_lora_styleB/tokenizer.json')

In [13]:
!cp -r phi3p5_lora_styleA /content/drive/MyDrive/Colab_Models/phi3p5_lora/
!cp -r phi3p5_lora_styleB /content/drive/MyDrive/Colab_Models/phi3p5_lora/

In [15]:
from transformers import GenerationConfig
from peft import PeftModel

# Load the fine-tuned model (Style A)
model_style_a = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto"
)
model_style_a = PeftModel.from_pretrained(model_style_a, output_dir_a)
model_style_a.eval()

# Example description
test_description = "A cozy cottage in the countryside with a private garden and scenic mountain views."

# Construct an inference prompt resembling style A:
prompt = f"""
Generate an appealing Airbnb listing title based on the description below.

Description:
{test_description}

Title:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

generation_config = GenerationConfig(
    max_new_tokens=20,
    temperature=0.7,
    do_sample=True,
)

with torch.no_grad():
    outputs = model_style_a.generate(
        **inputs,
        **generation_config.__dict__
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("==== GENERATED TEXT ====")
print(generated_text)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


==== GENERATED TEXT ====

Generate an appealing Airbnb listing title based on the description below.

Description:
A cozy cottage in the countryside with a private garden and scenic mountain views.

Title:
Cottage in Hertfordshire with mountain views. 
# Question

G


In [18]:
from transformers import GenerationConfig
from peft import PeftModel

# Load the fine-tuned model (Style B)
model_style_b = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto"
)
model_style_b = PeftModel.from_pretrained(model_style_b, output_dir_b)
model_style_b.eval()

# List of descriptions
test_descriptions = [
    "Modern condo in downtown with rooftop pool and gym access",
    "Cozy treehouse nestled in the forest with hiking trails nearby",
    "Beachfront villa with private dock and sunset views",
    "Spacious city loft near trendy cafes and nightlife",
    "Rustic cabin surrounded by wildlife, perfect for digital detox"
]

# Generation configuration
generation_config = GenerationConfig(
    max_new_tokens=20,  # Set max number of new tokens per title
    temperature=0.7,    # Sampling temperature (higher = more random)
    do_sample=True,     # Enable sampling for more diverse results
)

# Generate a title for each description
generated_titles = []

for description in test_descriptions:
    prompt = f"""
    Generate an appealing Airbnb listing title based on the description below.

    Description:
    {description}

    Title:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model_style_b.generate(
            **inputs,
            **generation_config.__dict__
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated title (removing the prompt portion)
    generated_title = generated_text.split("Title:")[-1].strip()

    generated_titles.append(generated_title)

# Print the results
print("==== GENERATED TITLES ====")
for i, title in enumerate(generated_titles):
    print(f"Description {i+1}: {test_descriptions[i]}")
    print(f"Generated Title {i+1}: {title}\n")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `ma

==== GENERATED TITLES ====
Description 1: Modern condo in downtown with rooftop pool and gym access
Generated Title 1: 5 Star New York City 3 bedroom condo in the heart of the city! 2

Description 2: Cozy treehouse nestled in the forest with hiking trails nearby
Generated Title 2: 1 bedroom treehouse in the forest, close to Notting Hill Gate and Holland Park tube

Description 3: Beachfront villa with private dock and sunset views
Generated Title 3: 🌴🎓🌺 Stunning beachfront 1 bedroom

Description 4: Spacious city loft near trendy cafes and nightlife
Generated Title 4: 1 Bedroom Loft near Trendy Areas in East London. Lots of Parks

Description 5: Rustic cabin surrounded by wildlife, perfect for digital detox
Generated Title 5: 🌳 🌲 🌳 🌲

